In [1]:
import numpy as np
import pandas as pd
import sklearn.metrics as mtr
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook
import datetime
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn import preprocessing

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 150)

In [2]:
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import Callback, EarlyStopping
from keras.layers import Dropout, PReLU, BatchNormalization
from keras.optimizers import Adam

Using TensorFlow backend.


In [3]:
#/kaggle/input/nfl-big-data-bowl-2020/train.csv
train = pd.read_csv('../data/train.csv', usecols=['GameId', 'PlayId', 'Team', 'X', 'Y', 'S', 'A', 'Dis', 'Turf',
                                                                               'Orientation', 'Dir', 'NflId', 'DisplayName', 'YardLine',
                                                                               'Quarter', 'GameClock', 'PossessionTeam', 'Down', 'Distance',
                                                                               'FieldPosition', 'NflIdRusher', 'OffenseFormation', 
                                                                               'OffensePersonnel', 'DefendersInTheBox', 'DefensePersonnel', 
                                                                               'PlayDirection', 'TimeHandoff', 'TimeSnap', 'Yards', 'Turf'])

In [4]:
outcomes = train[['GameId','PlayId','Yards']].drop_duplicates()

In [7]:
df_train = train[train.NflId==train.NflIdRusher].copy()

df_train['YardsFromOwnGoal'] = np.where(df_train.FieldPosition == df_train.PossessionTeam,
                                   df_train.YardLine, 50 + (50-df_train.YardLine))

df_train[['prev_game', 'prev_play', 'prev_team', 'prev_yfog']] = df_train[
         ['GameId', 'PlayId', 'Team', 'YardsFromOwnGoal']].shift(1)

filt = (df_train.GameId==df_train.prev_game) & (df_train.Team==df_train.prev_team) & (df_train.PlayId-df_train.prev_play<30)
df_train.loc[filt,'est_prev_yards'] = df_train[filt]['YardsFromOwnGoal'] - df_train[filt]['prev_yfog']


In [9]:
df_train.drop(['Team', 'X', 'Y', 'S', 'A', 'Dis', 'Turf',
               'Orientation', 'Dir', 'NflId', 'DisplayName', 'YardLine',
               'Quarter', 'GameClock', 'PossessionTeam', 'Down', 'Distance',
               'FieldPosition', 'NflIdRusher', 'OffenseFormation', 
               'OffensePersonnel', 'DefendersInTheBox', 'DefensePersonnel', 
               'PlayDirection', 'TimeHandoff', 'TimeSnap', 'Yards', 'Turf'], axis=1, inplace=True)

In [10]:
df_train.head()

,GameId,PlayId,YardsFromOwnGoal,prev_game,prev_play,prev_team,prev_yfog,est_prev_yards
18,2017090700,20170907000118,35,NaN,NaN,NaN,NaN,NaN
40,2017090700,20170907000139,43,2.017091e+09,2.017091e+13,home,35.0,8.0
62,2017090700,20170907000189,65,2.017091e+09,2.017091e+13,home,43.0,NaN
84,2017090700,20170907000345,98,2.017091e+09,2.017091e+13,home,65.0,NaN
98,2017090700,20170907000395,25,2.017091e+09,2.017091e+13,home,98.0,NaN


In [67]:
df_train['est_prev_yards'].value_counts().head()

2.0    612
4.0    595
3.0    586
5.0    544
1.0    470
Name: est_prev_yards, dtype: int64

## Features Engineering

In [11]:
#https://www.kaggle.com/rooshroosh/fork-of-neural-networks-different-architecture
def strtoseconds(txt):
    txt = txt.split(':')
    ans = int(txt[0])*60 + int(txt[1]) + int(txt[2])/60
    return ans

def strtofloat(x):
    try:
        return float(x)
    except:
        return -1

def map_weather(txt):
    ans = 1
    if pd.isna(txt):
        return 0
    if 'partly' in txt:
        ans*=0.5
    if 'climate controlled' in txt or 'indoor' in txt:
        return ans*3
    if 'sunny' in txt or 'sun' in txt:
        return ans*2
    if 'clear' in txt:
        return ans
    if 'cloudy' in txt:
        return -ans
    if 'rain' in txt or 'rainy' in txt:
        return -2*ans
    if 'snow' in txt:
        return -3*ans
    return 0

def map_turf(Turf):
    if Turf in ('Field Turf','A-Turf Titan','UBU Sports Speed S5-M','Artificial','DD GrassMaster',
                'UBU Speed Series-S5-M','FieldTurf','FieldTurf 360','Artifical','FieldTurf360',
                'Field turf','SISGrass','Twenty-Four/Seven Turf'):
        return 'Artificial'
    #elif Turf in ('natural grass','Naturall Grass','Natural grass','grass','Natural','Natural Grass','Grass'):
    #    return 'Natural'
    else:
        return 'Natural'

def OffensePersonnelSplit(x):
    dic = {'DB' : 0, 'DL' : 0, 'LB' : 0, 'OL' : 0, 'QB' : 0, 'RB' : 0, 'TE' : 0, 'WR' : 0}
    for xx in x.split(","):
        xxs = xx.split(" ")
        dic[xxs[-1]] = int(xxs[-2])
    return dic

def DefensePersonnelSplit(x):
    dic = {'DB' : 0, 'DL' : 0, 'LB' : 0, 'OL' : 0, 'RB' : 0}
    for xx in x.split(","):
        xxs = xx.split(" ")
        dic[xxs[-1]] = int(xxs[-2])
    return dic

def orientation_to_cat(x):
    x = np.clip(x, 0, 360 - 1)
    try:
        return str(int(x/15))
    except:
        return "nan"

## Functions for anchoring offense moving left from {0,0}

In [38]:
def create_features(df, deploy=False):
    def new_X(x_coordinate, play_direction):
        if play_direction == 'left':
            return 120.0 - x_coordinate
        else:
            return x_coordinate

    def new_Y(y_coordinate, play_direction):
        if play_direction == 'left':
            return (160 / 3) - y_coordinate
        else:
            return y_coordinate

    def new_line(rush_team, field_position, yardline):
        if rush_team == field_position:
            # offense starting at X = 0 plus the 10 yard endzone plus the line of scrimmage
            return 10.0 + yardline
        else:
            # half the field plus the yards between midfield and the line of scrimmage
            return 60.0 + (50 - yardline)

    def new_orientation(angle, play_direction):
        if play_direction == 'left':
            #new_angle = np.mod(180 + angle, 360)
            new_angle = 360.0 - angle
            if new_angle == 360.0:
                new_angle = 0.0
            return new_angle
        else:
            return angle

    def euclidean_distance(x1,y1,x2,y2):
        x_diff = (x1-x2)**2
        y_diff = (y1-y2)**2

        return np.sqrt(x_diff + y_diff)

    def back_direction(orientation):
        if orientation > 180.0:
            return 1
        else:
            return 0

    def update_yardline(df):
        new_yardline = df[df['NflId'] == df['NflIdRusher']]
        new_yardline['YardLine'] = new_yardline[['PossessionTeam','FieldPosition','YardLine']].apply(lambda x: new_line(x[0],x[1],x[2]), axis=1)
        new_yardline = new_yardline[['GameId','PlayId','YardLine']]

        return new_yardline

    def update_orientation(df, yardline):
        df['X'] = df[['X','PlayDirection']].apply(lambda x: new_X(x[0],x[1]), axis=1)
        df['Y'] = df[['Y','PlayDirection']].apply(lambda x: new_Y(x[0],x[1]), axis=1)
        df['Orientation'] = df[['Orientation','PlayDirection']].apply(lambda x: new_orientation(x[0],x[1]), axis=1)
        df['Dir'] = df[['Dir','PlayDirection']].apply(lambda x: new_orientation(x[0],x[1]), axis=1)

        df = df.drop('YardLine', axis=1)
        df = pd.merge(df, yardline, on=['GameId','PlayId'], how='inner')

        return df

    def back_features(df):
        carriers = df[df['NflId'] == df['NflIdRusher']][['GameId','PlayId','NflIdRusher','X','Y','Orientation','Dir','YardLine']]
        carriers['back_from_scrimmage'] = carriers['YardLine'] - carriers['X']
        carriers['back_oriented_down_field'] = carriers['Orientation'].apply(lambda x: back_direction(x))
        carriers['back_moving_down_field'] = carriers['Dir'].apply(lambda x: back_direction(x))
        carriers = carriers.rename(columns={'X':'back_X',
                                            'Y':'back_Y'})
        carriers = carriers[['GameId','PlayId','NflIdRusher','back_X','back_Y','back_from_scrimmage','back_oriented_down_field','back_moving_down_field']]

        return carriers

    def features_relative_to_back(df, carriers):
        player_distance = df[['GameId','PlayId','NflId','X','Y']]
        player_distance = pd.merge(player_distance, carriers, on=['GameId','PlayId'], how='inner')
        player_distance = player_distance[player_distance['NflId'] != player_distance['NflIdRusher']]
        player_distance['dist_to_back'] = player_distance[['X','Y','back_X','back_Y']].apply(lambda x: euclidean_distance(x[0],x[1],x[2],x[3]), axis=1)

        player_distance = player_distance.groupby(['GameId','PlayId','back_from_scrimmage','back_oriented_down_field','back_moving_down_field'])\
                                         .agg({'dist_to_back':['min','max','mean','std']})\
                                         .reset_index()
        player_distance.columns = ['GameId','PlayId','back_from_scrimmage','back_oriented_down_field','back_moving_down_field',
                                   'min_dist','max_dist','mean_dist','std_dist']

        return player_distance

    def defense_features(df):
        rusher = df[df['NflId'] == df['NflIdRusher']][['GameId','PlayId','Team','X','Y']]
        rusher.columns = ['GameId','PlayId','RusherTeam','RusherX','RusherY']

        defense = pd.merge(df,rusher,on=['GameId','PlayId'],how='inner')
        defense = defense[defense['Team'] != defense['RusherTeam']][['GameId','PlayId','X','Y','RusherX','RusherY']]
        defense['def_dist_to_back'] = defense[['X','Y','RusherX','RusherY']].apply(lambda x: euclidean_distance(x[0],x[1],x[2],x[3]), axis=1)

        defense = defense.groupby(['GameId','PlayId'])\
                         .agg({'def_dist_to_back':['min','max','mean','std']})\
                         .reset_index()
        defense.columns = ['GameId','PlayId','def_min_dist','def_max_dist','def_mean_dist','def_std_dist']

        return defense

    def static_features(df):
        static_features = df[df['NflId'] == df['NflIdRusher']][['GameId','PlayId','X','Y','S','A','Dis','Orientation','Dir',
                                                                'YardLine','Quarter','Down','Distance','DefendersInTheBox',
                                                                'Turf']].drop_duplicates()
        static_features['DefendersInTheBox'] = static_features['DefendersInTheBox'].fillna(np.mean(static_features['DefendersInTheBox']))

        return static_features


    def combine_features(relative_to_back, defense, static, deploy=deploy):
        df = pd.merge(relative_to_back,defense,on=['GameId','PlayId'],how='inner')
        df = pd.merge(df,static,on=['GameId','PlayId'],how='inner')

        if not deploy:
            df = pd.merge(df, outcomes, on=['GameId','PlayId'], how='inner')

        return df

    # Turf
    df['Turf'] = df['Turf'].apply(map_turf)

    ## WindSpeed
    #df['WindSpeed'] = df['WindSpeed'].apply(strtofloat)
    
    yardline = update_yardline(df)
    df = update_orientation(df, yardline)
    back_feats = back_features(df)
    rel_back = features_relative_to_back(df, back_feats)
    def_feats = defense_features(df)
    static_feats = static_features(df)
    basetable = combine_features(rel_back, def_feats, static_feats, deploy=deploy)
    
    ## OffensePersonnel
    #temp = basetable["OffensePersonnel"].apply(lambda x : pd.Series(OffensePersonnelSplit(x)))
    #temp.columns = ["Offense" + c for c in temp.columns]
    #temp["GameId"] = basetable["GameId"]
    #temp["PlayId"] = basetable["PlayId"]
    #basetable = basetable.merge(temp, on=['GameId','PlayId'], how='inner')

    ## DefensePersonnel
    #temp = basetable["DefensePersonnel"].apply(lambda x : pd.Series(DefensePersonnelSplit(x)))
    #temp.columns = ["Defense" + c for c in temp.columns]
    #temp["GameId"] = basetable["GameId"]
    #temp["PlayId"] = basetable["PlayId"]
    #basetable = basetable.merge(temp, on=['GameId','PlayId'], how='inner')

    ## Weather
    #basetable['GameWeather'] = basetable['GameWeather'].str.lower()
    #basetable['GameWeather'] = basetable['GameWeather'].apply(lambda x: "indoor" if not pd.isna(x) and "indoor" in x else x)
    #basetable['GameWeather'] = basetable['GameWeather'].apply(lambda x: x.replace('coudy', 'cloudy').replace('clouidy', 'cloudy').replace('party', 'partly') if not pd.isna(x) else x)
    #basetable['GameWeather'] = basetable['GameWeather'].apply(lambda x: x.replace('clear and sunny', 'sunny and clear') if not pd.isna(x) else x)
    #basetable['GameWeather'] = basetable['GameWeather'].apply(lambda x: x.replace('skies', '').replace("mostly", "").strip() if not pd.isna(x) else x)
    #basetable['GameWeather'] = basetable['GameWeather'].apply(map_weather)    

    #basetable['GameClock'] = basetable['GameClock'].apply(strtoseconds)
    
    #basetable = basetable.drop(['OffensePersonnel','DefensePersonnel'], axis=1)
    #basetable['WindSpeed'].fillna(basetable['WindSpeed'].mean(),inplace=True)

    #basetable['prev_game'].fillna(0,inplace=True)
    #basetable['prev_play'].fillna(0,inplace=True)
    #basetable['prev_team'].fillna('home',inplace=True)
    
    # Cria um label encoder object
    le = preprocessing.LabelEncoder()

    # Iteracao para cada coluna do dataset de treino
    for col in basetable:
        if basetable[col].dtype == 'object':
            le.fit_transform(basetable[col].astype(str))
            basetable[col] = le.transform(basetable[col])   
    
    return basetable

In [39]:
%time train_basetable = create_features(train, False)

CPU times: user 1min 25s, sys: 969 ms, total: 1min 26s
Wall time: 1min 20s


In [40]:
train_basetable = pd.merge(train_basetable, df_train, on=['GameId','PlayId'], how='inner')

In [41]:
train_basetable.fillna(0,inplace=True)

In [42]:
train_basetable.head()

,GameId,PlayId,back_from_scrimmage,back_oriented_down_field,back_moving_down_field,min_dist,max_dist,mean_dist,std_dist,def_min_dist,def_max_dist,def_mean_dist,def_std_dist,X,Y,S,A,Dis,Orientation,Dir,YardLine,Quarter,Down,Distance,DefendersInTheBox,Turf,Yards,YardsFromOwnGoal,prev_game,prev_play,prev_team,prev_yfog,est_prev_yards
0,2017090700,20170907000118,-33.75,0,1,1.449724,22.415872,8.046559,4.873845,4.593310,22.415872,9.752491,5.327299,78.75,30.53,3.63,3.35,0.38,161.98,245.74,45.0,1,3,2,6.0,0,8,35,0.000000e+00,0.000000e+00,0,0.0,0.0
1,2017090700,20170907000139,-18.07,1,1,0.792023,23.025872,8.614623,5.598683,4.287773,23.025872,10.297028,5.833217,71.07,27.16,3.06,2.41,0.34,210.70,312.20,53.0,1,1,10,6.0,0,3,43,2.017091e+09,2.017091e+13,home,35.0,8.0
2,2017090700,20170907000189,26.34,0,1,1.646390,20.726285,8.482583,4.642121,4.221670,20.726285,9.903689,5.073290,48.66,19.11,5.77,2.42,0.60,140.82,221.96,75.0,1,1,10,7.0,0,5,65,2.017091e+09,2.017091e+13,home,43.0,0.0
3,2017090700,20170907000345,92.47,1,1,0.918096,9.791231,5.549379,1.983128,4.528002,9.791231,6.309354,1.834174,15.53,25.36,4.45,3.20,0.46,186.22,275.44,108.0,1,2,2,9.0,0,2,98,2.017091e+09,2.017091e+13,home,65.0,0.0
4,2017090700,20170907000395,5.01,0,0,0.502892,21.214806,9.168819,5.611232,4.288088,21.214806,11.056456,5.900009,29.99,27.12,3.90,2.53,0.44,34.27,157.92,35.0,1,1,10,7.0,0,7,25,2.017091e+09,2.017091e+13,home,98.0,0.0


# Let's split our data into train/val

In [43]:
X = train_basetable.copy()
yards = X.Yards

y = np.zeros((yards.shape[0], 199))
for idx, target in enumerate(list(yards)):
    y[idx][99 + target] = 1

X.drop(['GameId','PlayId','Yards', 'prev_game', 'prev_play', 'prev_team'], axis=1, inplace=True)

In [44]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [58]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.10, random_state=12346)

In [59]:
print(X_train.shape, X_val.shape)
print(y_train.shape, y_val.shape)

(20853, 27) (2318, 27)
(20853, 199) (2318, 199)


# Let's build NN

Below class Metric based entirely on: https://www.kaggle.com/kingychiu/keras-nn-starter-crps-early-stopping
<br></br>
Below early stopping entirely based on: https://www.kaggle.com/c/nfl-big-data-bowl-2020/discussion/112868#latest-656533

In [60]:
class Metric(Callback):
    def __init__(self, model, callbacks, data):
        super().__init__()
        self.model = model
        self.callbacks = callbacks
        self.data = data

    def on_train_begin(self, logs=None):
        for callback in self.callbacks:
            callback.on_train_begin(logs)

    def on_train_end(self, logs=None):
        for callback in self.callbacks:
            callback.on_train_end(logs)

    def on_epoch_end(self, batch, logs=None):
        X_train, y_train = self.data[0][0], self.data[0][1]
        y_pred = self.model.predict(X_train)
        y_true = np.clip(np.cumsum(y_train, axis=1), 0, 1)
        y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1)
        tr_s = ((y_true - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * X_train.shape[0])
        tr_s = np.round(tr_s, 6)
        logs['tr_CRPS'] = tr_s

        X_valid, y_valid = self.data[1][0], self.data[1][1]

        y_pred = self.model.predict(X_valid)
        y_true = np.clip(np.cumsum(y_valid, axis=1), 0, 1)
        y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1)
        val_s = ((y_true - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * X_valid.shape[0])
        val_s = np.round(val_s, 6)
        logs['val_CRPS'] = val_s
        print('tr CRPS', tr_s, 'val CRPS', val_s)

        for callback in self.callbacks:
            callback.on_epoch_end(batch, logs)

In [61]:
#from keras.layers.advanced_activations import LeakyReLU, PReLU
model = Sequential()
model.add(Dense(512, input_dim=X.shape[1], activation='relu'))
#model.add(PReLU())
model.add(Dropout(0.4))

model.add(Dense(256, activation='relu'))
#model.add(PReLU())
model.add(Dropout(0.4))

model.add(Dense(256, activation='relu'))
#model.add(PReLU())
model.add(Dropout(0.2))

model.add(Dense(199, activation='softmax'))

In [62]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 512)               14336     
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_8 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 256)               65792     
_________________________________________________________________
dropout_9 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 199)              

In [63]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=[])

In [64]:
es = EarlyStopping(monitor='val_CRPS', 
                   mode='min',
                   restore_best_weights=True, 
                   verbose=1, 
                   patience=15)
es.set_model(model)

In [65]:
metric = Metric(model, [es], [(X_train,y_train), (X_val,y_val)])

In [66]:
%%time
model.fit(X_train, y_train, callbacks=[metric], epochs=250, batch_size=1024)

Epoch 1/250
20853/20853 [==============================] - 1s 34us/step - loss: 3.8740
tr CRPS 0.014181 val CRPS 0.014521
Epoch 2/250
20853/20853 [==============================] - 0s 24us/step - loss: 3.0258
tr CRPS 0.01387 val CRPS 0.014175
Epoch 3/250
20853/20853 [==============================] - 1s 24us/step - loss: 2.9400
tr CRPS 0.013699 val CRPS 0.013982
Epoch 4/250
20853/20853 [==============================] - 1s 24us/step - loss: 2.9092
tr CRPS 0.013566 val CRPS 0.013851
Epoch 5/250
20853/20853 [==============================] - 0s 24us/step - loss: 2.8841
tr CRPS 0.013502 val CRPS 0.013777
Epoch 6/250
20853/20853 [==============================] - 0s 24us/step - loss: 2.8676
tr CRPS 0.013438 val CRPS 0.013714
Epoch 7/250
20853/20853 [==============================] - 0s 24us/step - loss: 2.8507
tr CRPS 0.013393 val CRPS 0.013665
Epoch 8/250
20853/20853 [==============================] - 1s 25us/step - loss: 2.8365
tr CRPS 0.013344 val CRPS 0.013618
Epoch 9/250
20853/20853 [

20853/20853 [==============================] - 1s 26us/step - loss: 2.6161
tr CRPS 0.012247 val CRPS 0.013351
Epoch 69/250
20853/20853 [==============================] - 1s 27us/step - loss: 2.6118
tr CRPS 0.012263 val CRPS 0.013377
Epoch 70/250
20853/20853 [==============================] - 1s 28us/step - loss: 2.6122
tr CRPS 0.012258 val CRPS 0.013424
Epoch 71/250
20853/20853 [==============================] - 1s 26us/step - loss: 2.6037
tr CRPS 0.012201 val CRPS 0.013387
Epoch 72/250
20853/20853 [==============================] - 1s 26us/step - loss: 2.5986
tr CRPS 0.012174 val CRPS 0.013342
Epoch 73/250
20853/20853 [==============================] - 1s 26us/step - loss: 2.5938
tr CRPS 0.012168 val CRPS 0.013392
Epoch 74/250
20853/20853 [==============================] - 1s 27us/step - loss: 2.5911
tr CRPS 0.012164 val CRPS 0.013395
Epoch 75/250
20853/20853 [==============================] - 1s 26us/step - loss: 2.5933
tr CRPS 0.012145 val CRPS 0.013391
Epoch 76/250
20853/20853 [===

In [ ]:
'''Parametros:
1. adicionado as features TURF e WINDSPEED
2. aplicado Label Encoder para Turf
3. transformado Windspeed em Float e valores missing para MEAN
4 mudando test_size=0.10
5. correção da variavel Y
loss: 2.4766
tr CRPS 0.011503 val CRPS 0.012187'''

'''Parametros:
1. adicionado as features TURF e WINDSPEED
2. aplicado Label Encoder para Turf
3. transformado Windspeed em Float e valores missing para MEAN
4 mudando test_size=0.10
5. correção da variavel Y
loss: 2.5653
tr CRPS 0.012021 val CRPS 0.012252'''


'''Parametros:
1. adicionado as features TURF e WINDSPEED
2. aplicado Label Encoder para Turf
3. transformado Windspeed em Float e valores missing para MEAN
4 mudando test_size=0.10
5. correção da variavel Y
6. model.add(PReLU())
loss: 2.5095
tr CRPS 0.011781 val CRPS 0.012269'''


'''Parametros:
1. adicionado as features TURF e WINDSPEED
2. aplicado Label Encoder para Turf
3. transformado Windspeed em Float e valores missing para MEAN
4 mudando test_size=0.10
5. correção da variavel Y
6. LeakyReLU(alpha=.001, weights=None) 
loss: 2.4869
tr CRPS 0.011677 val CRPS 0.012281'''

'''Parametros:
1. adicionado as features TURF e WINDSPEED
2. aplicado Label Encoder para Turf
3. transformado Windspeed em Float e valores missing para MEAN
4 mudando test_size=0.10
5. correção da variavel Y
6. model.add(PReLU())
loss: 2.4001
tr CRPS 0.011056 val CRPS 0.012336'''


'''Parametros:
1. adicionado as features TURF
2. aplicado Label Encoder para Turf
3. mudando test_size=0.10
4. random_state=12346
5. correção da variavel Y
loss: 2.5570
tr CRPS 0.011883 val CRPS 0.013385'''

In [ ]:
class GP:
    def __init__(self):
        self.classes = 20
        self.class_names = [ 'class_0',
                             'class_1',
                             'class_2',
                             'class_3',
                             'class_4',
                             'class_5',
                             'class_6',
                             'class_7',
                             'class_8',
                             'class_9',
                             'class_10',
                             'class_11',
                             'class_12',
                             'class_13',
                            'class_14',
                            'class_15',
                            'class_16',
                            'class_17',
                            'class_18',
                            'class_19',
                           ]


    def GrabPredictions(self, data):
        oof_preds = np.zeros((len(data), len(self.class_names)))
        oof_preds[:,0] = self.GP_class_0(data)
        oof_preds[:,1] = self.GP_class_1(data)
        oof_preds[:,2] = self.GP_class_2(data)
        oof_preds[:,3] = self.GP_class_3(data)
        oof_preds[:,4] = self.GP_class_4(data)
        oof_preds[:,5] = self.GP_class_5(data)
        oof_preds[:,6] = self.GP_class_6(data)
        oof_preds[:,7] = self.GP_class_7(data)
        oof_preds[:,8] = self.GP_class_8(data)
        oof_preds[:,9] = self.GP_class_9(data)
        oof_preds[:,10] = self.GP_class_10(data)
        oof_preds[:,11] = self.GP_class_11(data)
        oof_preds[:,12] = self.GP_class_12(data)
        oof_preds[:,13] = self.GP_class_13(data)
        oof_preds[:,14] = self.GP_class_14(data)
        oof_preds[:,15] = self.GP_class_15(data)
        oof_preds[:,16] = self.GP_class_16(data)
        oof_preds[:,17] = self.GP_class_17(data)
        oof_preds[:,18] = self.GP_class_18(data)
        oof_preds[:,19] = self.GP_class_19(data)
        oof_df = pd.DataFrame(np.exp(oof_preds), columns=self.class_names)
        oof_df =oof_df.div(oof_df.sum(axis=1), axis=0)
        
        return oof_df.values


    def GP_class_0(self,data):
        return(0.250000*np.tanh(((((((data[:,0]) - (((data[:,0]) + (data[:,0]))))) + (data[:,0]))) + (((((data[:,0]) * 2.0)) * 2.0)))) +
                0.250000*np.tanh(((((((((((((data[:,2]) - (data[:,7]))) * 2.0)) + (data[:,7]))) * 2.0)) + (data[:,0]))) + (((((data[:,2]) + (data[:,0]))) - (data[:,7]))))) +
                0.250000*np.tanh(((((((((((((((((((data[:,3]) - (data[:,7]))) - (((data[:,0]) / 2.0)))) + (data[:,0]))) - (data[:,7]))) + (data[:,2]))) + (((data[:,7]) + (data[:,2]))))) + (data[:,7]))) - (data[:,7]))) + (data[:,0]))) +
                0.250000*np.tanh(((((((((data[:,0]) - (data[:,7]))) - (data[:,14]))) + (((data[:,17]) - (data[:,14]))))) + (((((((data[:,0]) - ((1.0)))) - (data[:,0]))) + (data[:,20]))))) +
                0.250000*np.tanh(((((data[:,18]) + (((((((data[:,0]) - (data[:,7]))) + (data[:,2]))) + (data[:,0]))))) - (((data[:,7]) + (data[:,2]))))) +
                0.250000*np.tanh((((((((data[:,8]) + ((((((((data[:,13]) + (data[:,3]))) + (data[:,0]))/2.0)) + ((((((((data[:,0]) + (((((data[:,0]) - (data[:,0]))) / 2.0)))) + (data[:,17]))/2.0)) + (data[:,11]))))))/2.0)) + (data[:,0]))) - (data[:,13]))) +
                0.250000*np.tanh(((((data[:,0]) * 2.0)) + (((((((((data[:,2]) + (data[:,2]))) * 2.0)) - (((data[:,14]) - (((np.tanh((data[:,2]))) * 2.0)))))) - (data[:,7]))))) +
                0.250000*np.tanh(((((((((((((data[:,17]) - (((data[:,17]) * (((((data[:,17]) * (data[:,17]))) * 2.0)))))) + (((((((data[:,17]) * 2.0)) * 2.0)) * 2.0)))/2.0)) / 2.0)) + (((data[:,17]) - ((7.76236486434936523)))))/2.0)) + (((((((data[:,17]) * (data[:,17]))) * 2.0)) * 2.0)))) +
                0.250000*np.tanh(((((((((((data[:,17]) + (data[:,17]))/2.0)) + ((((data[:,17]) + ((((data[:,17]) + (data[:,17]))/2.0)))/2.0)))/2.0)) * (data[:,17]))) - (data[:,13]))) +
                0.250000*np.tanh((((((data[:,19]) + (((((((((data[:,19]) * 2.0)) + (data[:,19]))) - ((1.07645297050476074)))) + (((data[:,22]) + (data[:,22]))))))/2.0)) + (((data[:,22]) + (data[:,4]))))))
    
    def GP_class_1(self,data):
        return(0.250000*np.tanh(((((((((((((data[:,3]) + (((np.tanh((data[:,2]))) - (data[:,14]))))) - ((10.0)))) - ((4.88989353179931641)))) - ((13.92175483703613281)))) / 2.0)) - (data[:,6]))) +
                0.250000*np.tanh(data[:,3]) +
                0.250000*np.tanh(((data[:,2]) + (((((np.tanh((((data[:,14]) * 2.0)))) - (((((((data[:,7]) * 2.0)) - (((((data[:,0]) - (data[:,14]))) - (data[:,7]))))) - (data[:,0]))))) + (data[:,0]))))) +
                0.250000*np.tanh(((((((data[:,0]) - ((-1.0*((data[:,14])))))) + ((((((((((data[:,0]) - (data[:,7]))) - ((((data[:,14]) + (data[:,14]))/2.0)))) + (((data[:,2]) - (((data[:,0]) * (data[:,14]))))))/2.0)) - (data[:,14]))))) - (data[:,7]))) +
                0.250000*np.tanh(((((data[:,0]) / 2.0)) + (data[:,0]))) +
                0.250000*np.tanh(((((data[:,19]) - (data[:,19]))) / 2.0)) +
                0.250000*np.tanh(((((np.tanh((((np.tanh((data[:,5]))) - ((((1.69341480731964111)) + ((((((np.tanh((data[:,14]))) - (data[:,14]))) + (data[:,14]))/2.0)))))))) - (data[:,2]))) - (((data[:,14]) * 2.0)))) +
                0.250000*np.tanh((((((data[:,22]) + (((data[:,0]) + ((((((((data[:,22]) / 2.0)) + ((-1.0*((data[:,13])))))/2.0)) + (data[:,19]))))))/2.0)) - (data[:,13]))) +
                0.250000*np.tanh(((((((data[:,17]) / 2.0)) * (data[:,17]))) * (((data[:,17]) + (((data[:,17]) * (((((((((data[:,17]) + (data[:,17]))) * (data[:,17]))) * (data[:,17]))) + (((np.tanh((data[:,17]))) / 2.0)))))))))) +
                0.250000*np.tanh((((((data[:,14]) * (((data[:,13]) + (data[:,3]))))) + (((data[:,14]) * (((data[:,14]) - ((-1.0*((data[:,3])))))))))/2.0)))
    
    def GP_class_2(self,data):
        return(0.250000*np.tanh(((data[:,0]) + (np.tanh((((((((((data[:,2]) + (((data[:,2]) / 2.0)))/2.0)) + (data[:,0]))/2.0)) * (data[:,7]))))))) +
                0.250000*np.tanh((((data[:,22]) + (data[:,22]))/2.0)) +
                0.250000*np.tanh((((((data[:,14]) + ((-1.0*((((((4.73206138610839844)) + (((((((((((np.tanh((data[:,7]))) + (data[:,1]))/2.0)) - (np.tanh((((((data[:,0]) - (data[:,20]))) - (data[:,14]))))))) * ((7.0)))) + (data[:,14]))/2.0)))/2.0))))))/2.0)) - (data[:,14]))) +
                0.250000*np.tanh(((((data[:,1]) + ((4.15603733062744141)))) / 2.0)) +
                0.250000*np.tanh(((((((((((((data[:,0]) - (data[:,7]))) - (data[:,7]))) + ((((data[:,0]) + (data[:,0]))/2.0)))) * 2.0)) + (data[:,0]))) * 2.0)) +
                0.250000*np.tanh(((((data[:,11]) / 2.0)) / 2.0)) +
                0.250000*np.tanh(((((((((((((((((((((((data[:,2]) / 2.0)) + (((data[:,2]) / 2.0)))/2.0)) / 2.0)) / 2.0)) / 2.0)) + ((((-1.0*((((data[:,0]) / 2.0))))) * (((data[:,0]) / 2.0)))))/2.0)) + (data[:,0]))/2.0)) + (np.tanh((data[:,0]))))/2.0)) / 2.0)) +
                0.250000*np.tanh(((((data[:,0]) + (((data[:,0]) + (((((((data[:,14]) * (data[:,14]))) + (data[:,14]))) - (data[:,0]))))))) - (data[:,14]))) +
                0.250000*np.tanh((((((np.tanh((np.tanh(((0.0)))))) * (data[:,15]))) + (data[:,20]))/2.0)) +
                0.250000*np.tanh(((data[:,14]) * ((((data[:,13]) + ((((data[:,13]) + ((((data[:,14]) + ((((((data[:,14]) + ((((data[:,14]) + (((data[:,13]) - ((((-1.0*(((((data[:,14]) + (data[:,14]))/2.0))))) * 2.0)))))/2.0)))) + (data[:,14]))/2.0)))/2.0)))/2.0)))/2.0)))))
    
    def GP_class_3(self,data):
        return(0.250000*np.tanh(((((((((8.0)) + (((((((5.33416414260864258)) + ((9.0)))/2.0)) + ((((8.0)) * 2.0)))))) * 2.0)) + ((5.33416414260864258)))/2.0)) +
                0.250000*np.tanh((((9.48088836669921875)) + (((((10.56953334808349609)) + ((((4.48959350585937500)) + (np.tanh(((((3.0)) + ((9.0)))))))))/2.0)))) +
                0.250000*np.tanh((((((((((data[:,22]) / 2.0)) + (data[:,22]))/2.0)) * 2.0)) / 2.0)) +
                0.250000*np.tanh((10.44883441925048828)) +
                0.250000*np.tanh(((((data[:,0]) + (((data[:,0]) + ((-1.0*((data[:,9])))))))) + (((((data[:,0]) - (data[:,7]))) - (data[:,9]))))) +
                0.250000*np.tanh(((np.tanh((data[:,6]))) - (data[:,21]))) +
                0.250000*np.tanh(((data[:,0]) - (((((data[:,14]) + (((((data[:,14]) + (((data[:,14]) + (((data[:,14]) - (data[:,0]))))))) - (((((data[:,0]) - (data[:,14]))) + (data[:,14]))))))) - (((((data[:,14]) * (data[:,14]))) + (data[:,18]))))))) +
                0.250000*np.tanh(((data[:,14]) * ((((data[:,13]) + (data[:,14]))/2.0)))) +
                0.250000*np.tanh((((((data[:,8]) * (((data[:,8]) / 2.0)))) + ((((((data[:,9]) * (data[:,8]))) + (data[:,8]))/2.0)))/2.0)) +
                0.250000*np.tanh((((data[:,2]) + (((((((data[:,2]) + (data[:,20]))/2.0)) + ((((((((data[:,2]) + (data[:,8]))/2.0)) / 2.0)) - (data[:,8]))))/2.0)))/2.0)))
    
    def GP_class_4(self,data):
        return(0.250000*np.tanh((((12.98819637298583984)) / 2.0)) +
                0.250000*np.tanh(((((4.75780344009399414)) + (((((((8.66014671325683594)) + ((4.18107128143310547)))/2.0)) * ((8.97841739654541016)))))/2.0)) +
                0.250000*np.tanh((((9.0)) / 2.0)) +
                0.250000*np.tanh(((((((((((10.0)) + (np.tanh((((((6.0)) + ((((13.80149555206298828)) + ((7.0)))))/2.0)))))) + ((10.0)))) + (((((((data[:,0]) + ((8.0)))) / 2.0)) / 2.0)))/2.0)) + ((((-1.0*((((data[:,2]) * (data[:,9])))))) * 2.0)))) +
                0.250000*np.tanh(((((data[:,0]) + (((((((data[:,22]) + (data[:,22]))/2.0)) + (data[:,22]))/2.0)))) + ((((((data[:,22]) + ((((((data[:,22]) * 2.0)) + (data[:,22]))/2.0)))) + (data[:,18]))/2.0)))) +
                0.250000*np.tanh(((((((((((((np.tanh((data[:,18]))) + ((((data[:,1]) + (data[:,18]))/2.0)))/2.0)) * (data[:,18]))) * (data[:,18]))) + (((data[:,18]) * (data[:,18]))))/2.0)) - (((((((np.tanh((data[:,18]))) + (((data[:,18]) * (data[:,18]))))/2.0)) + (data[:,17]))/2.0)))) +
                0.250000*np.tanh(((np.tanh(((((((np.tanh((data[:,6]))) - (((np.tanh((data[:,6]))) - (data[:,6]))))) + (np.tanh((data[:,6]))))/2.0)))) - ((((data[:,10]) + ((((data[:,2]) + (((np.tanh((data[:,6]))) - (((np.tanh((np.tanh((data[:,6]))))) - (data[:,10]))))))/2.0)))/2.0)))) +
                0.250000*np.tanh((((((((((((data[:,10]) + (data[:,4]))) + (((data[:,10]) + (((data[:,0]) * (data[:,10]))))))/2.0)) + (((data[:,4]) * (((data[:,10]) + (data[:,4]))))))/2.0)) + (data[:,7]))/2.0)) +
                0.250000*np.tanh((((((data[:,11]) + (((((data[:,18]) / 2.0)) * (((data[:,11]) * (data[:,18]))))))/2.0)) * (((data[:,18]) * (((((((((data[:,18]) * (((((((data[:,11]) / 2.0)) / 2.0)) * (data[:,18]))))) / 2.0)) / 2.0)) * ((-1.0*((data[:,21])))))))))) +
                0.250000*np.tanh((((-1.0*((((((((((((data[:,2]) - (data[:,2]))) * (np.tanh((data[:,2]))))) + (np.tanh((((data[:,14]) * 2.0)))))/2.0)) + (data[:,2]))/2.0))))) - (data[:,2]))))
    
    def GP_class_5(self,data):
        return(0.250000*np.tanh(((((((((((3.46574378013610840)) + ((((np.tanh(((3.46574378013610840)))) + ((8.46705245971679688)))/2.0)))/2.0)) * 2.0)) + ((((3.46574378013610840)) + ((4.0)))))) + ((((4.0)) * 2.0)))) +
                0.250000*np.tanh((((10.55856513977050781)) / 2.0)) +
                0.250000*np.tanh((((3.76695251464843750)) / 2.0)) +
                0.250000*np.tanh((((((6.0)) / 2.0)) + ((((8.57984828948974609)) * 2.0)))) +
                0.250000*np.tanh((((((((((((3.42168045043945312)) + (data[:,7]))) + (np.tanh((data[:,7]))))) + (np.tanh((np.tanh((np.tanh(((3.42168045043945312)))))))))/2.0)) + (data[:,7]))/2.0)) +
                0.250000*np.tanh(((((((((((data[:,6]) - (data[:,9]))) + (data[:,7]))/2.0)) * 2.0)) + ((((((data[:,7]) + (data[:,9]))/2.0)) - (data[:,9]))))/2.0)) +
                0.250000*np.tanh(((((((((((data[:,22]) + (data[:,22]))) + (data[:,11]))/2.0)) + (data[:,22]))/2.0)) / 2.0)) +
                0.250000*np.tanh((((((((((0.76044219732284546)) / 2.0)) + ((0.76044219732284546)))/2.0)) + (np.tanh((np.tanh((((((((0.76043862104415894)) / 2.0)) + ((1.0)))/2.0)))))))/2.0)) +
                0.250000*np.tanh(((((((data[:,12]) + (np.tanh((data[:,9]))))/2.0)) + (((((((((((data[:,12]) + (((((0.0)) + (np.tanh((((((((data[:,7]) + (data[:,6]))/2.0)) + (data[:,7]))/2.0)))))/2.0)))/2.0)) / 2.0)) / 2.0)) + (data[:,7]))/2.0)))/2.0)) +
                0.250000*np.tanh(((((((((data[:,20]) / 2.0)) / 2.0)) * (((np.tanh((data[:,20]))) * (((data[:,20]) / 2.0)))))) * ((((((data[:,20]) / 2.0)) + (((data[:,20]) / 2.0)))/2.0)))))
    
    def GP_class_6(self,data):
        return(0.250000*np.tanh(((((((11.68076229095458984)) + (((((11.68075847625732422)) + ((((11.47270107269287109)) + (((((11.47269725799560547)) + ((11.68076229095458984)))/2.0)))))/2.0)))/2.0)) + ((((4.0)) * 2.0)))) +
                0.250000*np.tanh((((8.0)) + ((((10.0)) + ((6.59042119979858398)))))) +
                0.250000*np.tanh(((((((((((13.33760547637939453)) + (((((3.86388397216796875)) + (((((((8.18321037292480469)) + (data[:,4]))) + (((((((5.95386552810668945)) + ((12.93883609771728516)))) + (((((10.73907089233398438)) + ((4.0)))/2.0)))/2.0)))/2.0)))/2.0)))/2.0)) + (np.tanh(((3.0)))))) - ((2.0)))) + (((((10.61559963226318359)) + (data[:,1]))/2.0)))) +
                0.250000*np.tanh(((np.tanh(((((((((((7.13513946533203125)) + ((12.48778533935546875)))/2.0)) + (((((((7.13513565063476562)) - ((8.12031841278076172)))) + (((((((((7.13513565063476562)) + ((((10.69830417633056641)) + ((10.69830799102783203)))))) + ((10.69830799102783203)))) + ((7.83078956604003906)))/2.0)))/2.0)))/2.0)) * 2.0)))) + ((3.0)))) +
                0.250000*np.tanh(((((((((((data[:,7]) - (data[:,0]))) * 2.0)) - (data[:,7]))) + (data[:,7]))) + (data[:,0]))) +
                0.250000*np.tanh(((((data[:,7]) / 2.0)) / 2.0)) +
                0.250000*np.tanh((((((((((((((((1.0)) + (((((((data[:,2]) + (((((((1.0)) * (data[:,2]))) + (data[:,14]))/2.0)))/2.0)) + ((1.0)))/2.0)))) + ((((1.0)) - (((data[:,2]) * (data[:,14]))))))/2.0)) - (data[:,2]))) * 2.0)) + (data[:,14]))/2.0)) + (data[:,14]))) +
                0.250000*np.tanh(((np.tanh((np.tanh(((((((data[:,19]) * ((0.0)))) + ((0.0)))/2.0)))))) / 2.0)) +
                0.250000*np.tanh((((((((0.0)) / 2.0)) / 2.0)) * (((((((((data[:,15]) / 2.0)) * ((0.0)))) / 2.0)) * ((((-1.0*(((0.0))))) / 2.0)))))) +
                0.250000*np.tanh(((((data[:,15]) * (((((((((((((np.tanh(((((0.09032609313726425)) / 2.0)))) / 2.0)) / 2.0)) / 2.0)) / 2.0)) / 2.0)) / 2.0)))) / 2.0)))
    
    def GP_class_7(self,data):
        return(0.250000*np.tanh((10.27210903167724609)) +
                0.250000*np.tanh((((((((((10.0)) + (((((((((8.26972770690917969)) + ((11.06334972381591797)))/2.0)) * 2.0)) / 2.0)))) + ((((((5.0)) + ((((5.0)) + ((7.92727756500244141)))))) * 2.0)))) * ((7.92728137969970703)))) + ((10.0)))) +
                0.250000*np.tanh((8.42519950866699219)) +
                0.250000*np.tanh(((((((data[:,14]) + (data[:,14]))) + ((((((1.59392631053924561)) + (data[:,14]))) + (data[:,14]))))) + (np.tanh((((((data[:,14]) + (((((((3.0)) + (((data[:,14]) * 2.0)))/2.0)) + ((1.59391915798187256)))))) + ((1.59392631053924561)))))))) +
                0.250000*np.tanh((((data[:,7]) + (((data[:,0]) + (((((((data[:,7]) + (data[:,4]))/2.0)) + (((data[:,7]) * 2.0)))/2.0)))))/2.0)) +
                0.250000*np.tanh((((((((((((((data[:,9]) - (((data[:,0]) - (data[:,0]))))) - (((data[:,0]) / 2.0)))) / 2.0)) - (((data[:,21]) - (data[:,7]))))) + (data[:,21]))/2.0)) - (data[:,0]))) +
                0.250000*np.tanh((((((((((2.0)) + ((((((data[:,21]) + ((((data[:,13]) + (((((((((((data[:,13]) * 2.0)) + (data[:,21]))/2.0)) + (data[:,13]))/2.0)) * 2.0)))/2.0)))/2.0)) - (np.tanh(((((0.0)) - (data[:,21]))))))))) + ((1.0)))/2.0)) + (data[:,13]))/2.0)) +
                0.250000*np.tanh(((((data[:,14]) / 2.0)) / 2.0)) +
                0.250000*np.tanh((((((((((((((((-1.0*(((((((data[:,21]) / 2.0)) + (data[:,8]))/2.0))))) / 2.0)) / 2.0)) / 2.0)) + (data[:,9]))/2.0)) / 2.0)) + ((((((((((((data[:,7]) / 2.0)) / 2.0)) / 2.0)) / 2.0)) + (data[:,7]))/2.0)))/2.0)) +
                0.250000*np.tanh(((((((np.tanh(((1.0)))) / 2.0)) / 2.0)) / 2.0)))
    
    def GP_class_8(self,data):
        return(0.250000*np.tanh(((((((data[:,0]) + (((((6.71804094314575195)) + (data[:,15]))/2.0)))) + ((((((11.56385326385498047)) * 2.0)) * ((((10.48986148834228516)) + ((11.56385326385498047)))))))) / 2.0)) +
                0.250000*np.tanh((((((((((data[:,14]) / 2.0)) + ((2.83755254745483398)))/2.0)) + ((((((data[:,12]) + (data[:,14]))/2.0)) + (data[:,14]))))) + ((((data[:,9]) + (((data[:,12]) + (data[:,14]))))/2.0)))) +
                0.250000*np.tanh(((((((((data[:,6]) + (((((((3.50821948051452637)) + ((((data[:,21]) + ((11.92932796478271484)))/2.0)))) + ((1.0)))/2.0)))) + (data[:,10]))/2.0)) + (data[:,21]))/2.0)) +
                0.250000*np.tanh(((data[:,14]) + (np.tanh((np.tanh(((-1.0*((((data[:,22]) - (data[:,13])))))))))))) +
                0.250000*np.tanh(((((1.0)) + ((((data[:,7]) + (data[:,7]))/2.0)))/2.0)) +
                0.250000*np.tanh(((((((((data[:,21]) / 2.0)) / 2.0)) * (((((((np.tanh((((np.tanh((data[:,13]))) / 2.0)))) / 2.0)) + (((((((np.tanh(((((data[:,21]) + (data[:,21]))/2.0)))) / 2.0)) - (data[:,12]))) / 2.0)))) / 2.0)))) - (data[:,12]))) +
                0.250000*np.tanh((((data[:,7]) + (np.tanh(((((data[:,12]) + (((data[:,7]) * ((((data[:,13]) + (data[:,13]))/2.0)))))/2.0)))))/2.0)) +
                0.250000*np.tanh(((((((data[:,13]) / 2.0)) / 2.0)) / 2.0)) +
                0.250000*np.tanh((((((((((np.tanh((data[:,9]))) + (data[:,13]))/2.0)) + ((((((((data[:,21]) / 2.0)) * 2.0)) + ((((3.94983267784118652)) / 2.0)))/2.0)))/2.0)) + ((((np.tanh(((3.94983267784118652)))) + (data[:,9]))/2.0)))/2.0)) +
                0.250000*np.tanh(((((((((((((((data[:,14]) * (((((((((data[:,19]) / 2.0)) * (np.tanh((((data[:,2]) / 2.0)))))) / 2.0)) / 2.0)))) / 2.0)) / 2.0)) / 2.0)) / 2.0)) / 2.0)) * (((((data[:,11]) / 2.0)) / 2.0)))))
    
    def GP_class_9(self,data):
        return(0.250000*np.tanh(((((((data[:,14]) * 2.0)) / 2.0)) + (((((data[:,14]) + (np.tanh((((data[:,14]) * 2.0)))))) + (data[:,14]))))) +
                0.250000*np.tanh(((data[:,9]) - (((((-1.0*((data[:,11])))) + (data[:,9]))/2.0)))) +
                0.250000*np.tanh(((data[:,21]) * 2.0)) +
                0.250000*np.tanh((((((data[:,14]) + (data[:,14]))/2.0)) + ((((data[:,13]) + ((((np.tanh((((data[:,9]) + ((6.0)))))) + (data[:,9]))/2.0)))/2.0)))) +
                0.250000*np.tanh(np.tanh(((1.0)))) +
                0.250000*np.tanh(((np.tanh((np.tanh(((((np.tanh((((data[:,20]) * (((((((np.tanh((((data[:,20]) / 2.0)))) + (data[:,12]))/2.0)) + ((((data[:,15]) + ((((data[:,17]) + (data[:,9]))/2.0)))/2.0)))/2.0)))))) + (data[:,9]))/2.0)))))) / 2.0)) +
                0.250000*np.tanh(((((data[:,2]) - (data[:,11]))) / 2.0)) +
                0.250000*np.tanh((((((((data[:,13]) / 2.0)) + (((data[:,7]) + (data[:,7]))))/2.0)) - (((((data[:,22]) + (((((data[:,14]) * (((((data[:,7]) * (((data[:,13]) - ((((data[:,7]) + (((((data[:,13]) / 2.0)) + (data[:,7]))))/2.0)))))) / 2.0)))) * 2.0)))) / 2.0)))) +
                0.250000*np.tanh(((((((((((((((data[:,1]) / 2.0)) + ((1.0)))/2.0)) + ((1.0)))/2.0)) + ((((((((((1.0)) / 2.0)) + (np.tanh(((1.0)))))/2.0)) + (np.tanh((((((1.0)) + ((1.0)))/2.0)))))/2.0)))/2.0)) + ((((data[:,20]) + ((1.0)))/2.0)))/2.0)) +
                0.250000*np.tanh(((data[:,1]) * ((((data[:,14]) + (np.tanh(((((0.0)) / 2.0)))))/2.0)))))
    
    def GP_class_10(self,data):
        return(0.250000*np.tanh((((((((((data[:,2]) + (((data[:,9]) * (np.tanh((data[:,14]))))))/2.0)) + ((((((data[:,14]) + (data[:,20]))/2.0)) + (data[:,0]))))/2.0)) + (data[:,14]))/2.0)) +
                0.250000*np.tanh((((((data[:,9]) + ((-1.0*((((((((2.04309988021850586)) + (((data[:,11]) + (((np.tanh((data[:,0]))) / 2.0)))))/2.0)) * 2.0))))))/2.0)) + (data[:,2]))) +
                0.250000*np.tanh(data[:,21]) +
                0.250000*np.tanh((((((((data[:,14]) + (np.tanh((((data[:,6]) / 2.0)))))/2.0)) + (data[:,14]))) + (((((data[:,14]) * 2.0)) + ((((data[:,13]) + (data[:,14]))/2.0)))))) +
                0.250000*np.tanh(data[:,9]) +
                0.250000*np.tanh(((((((data[:,15]) * 2.0)) / 2.0)) / 2.0)) +
                0.250000*np.tanh((((((((2.72836518287658691)) * ((((1.0)) / 2.0)))) / 2.0)) / 2.0)) +
                0.250000*np.tanh(((((np.tanh((((data[:,7]) + (np.tanh(((((((((((0.0)) / 2.0)) / 2.0)) / 2.0)) / 2.0)))))))) / 2.0)) / 2.0)) +
                0.250000*np.tanh(np.tanh((((np.tanh(((11.43236827850341797)))) / 2.0)))) +
                0.250000*np.tanh(np.tanh((((data[:,7]) / 2.0)))))
    
    def GP_class_11(self,data):
        return(0.250000*np.tanh((-1.0*((((((((((((((data[:,5]) - (((data[:,9]) - ((((-1.0*(((11.40053558349609375))))) / 2.0)))))) + ((((11.40053558349609375)) + ((((11.40053558349609375)) * 2.0)))))) / 2.0)) - (data[:,7]))) - ((-1.0*(((11.40053939819335938))))))) * 2.0))))) +
                0.250000*np.tanh(((((data[:,14]) * 2.0)) + (data[:,14]))) +
                0.250000*np.tanh(((data[:,2]) + (((((data[:,5]) + (data[:,3]))) + ((((data[:,2]) + (((data[:,3]) * 2.0)))/2.0)))))) +
                0.250000*np.tanh(((((((((((data[:,21]) + (data[:,21]))) + ((-1.0*(((((np.tanh((np.tanh((((data[:,14]) / 2.0)))))) + (data[:,21]))/2.0))))))/2.0)) * 2.0)) + ((((data[:,14]) + (data[:,3]))/2.0)))/2.0)) +
                0.250000*np.tanh(((((((data[:,0]) + ((((data[:,9]) + ((((data[:,9]) + (((((((((((((((data[:,9]) / 2.0)) - (data[:,10]))) + (data[:,5]))/2.0)) - (data[:,9]))) + (data[:,9]))/2.0)) / 2.0)))/2.0)))/2.0)))/2.0)) + (data[:,9]))/2.0)) +
                0.250000*np.tanh((((((((data[:,13]) + ((((data[:,13]) + (data[:,13]))/2.0)))) + (data[:,1]))/2.0)) / 2.0)) +
                0.250000*np.tanh(data[:,14]) +
                0.250000*np.tanh(((((data[:,2]) / 2.0)) / 2.0)) +
                0.250000*np.tanh((((data[:,9]) + (data[:,9]))/2.0)) +
                0.250000*np.tanh(data[:,2]))
    
    def GP_class_12(self,data):
        return(0.250000*np.tanh((((((((-1.0*((data[:,18])))) - (np.tanh((data[:,9]))))) + ((8.0)))) - ((14.74865913391113281)))) +
                0.250000*np.tanh((((((data[:,21]) + (np.tanh((data[:,9]))))) + ((((((((data[:,5]) / 2.0)) * 2.0)) + (data[:,5]))/2.0)))/2.0)) +
                0.250000*np.tanh(((((data[:,14]) + (data[:,14]))) + (((data[:,14]) * 2.0)))) +
                0.250000*np.tanh(((data[:,2]) + (((((((data[:,14]) + ((((np.tanh((data[:,10]))) + (data[:,17]))/2.0)))/2.0)) + (data[:,21]))/2.0)))) +
                0.250000*np.tanh((((data[:,9]) + ((((((((((((data[:,9]) + ((((((data[:,9]) / 2.0)) + (((data[:,9]) * 2.0)))/2.0)))/2.0)) + ((-1.0*((data[:,21])))))/2.0)) * (data[:,9]))) + (((((np.tanh((((data[:,21]) / 2.0)))) / 2.0)) / 2.0)))/2.0)))/2.0)) +
                0.250000*np.tanh(((((0.0)) + (data[:,13]))/2.0)) +
                0.250000*np.tanh((((((((data[:,2]) / 2.0)) * 2.0)) + (data[:,7]))/2.0)) +
                0.250000*np.tanh((((((data[:,14]) * 2.0)) + (np.tanh((data[:,14]))))/2.0)) +
                0.250000*np.tanh(np.tanh((((((((((-1.0*((((np.tanh((data[:,9]))) / 2.0))))) / 2.0)) / 2.0)) + (((data[:,3]) * (data[:,13]))))/2.0)))) +
                0.250000*np.tanh((-1.0*((((((((-1.0*(((((((-1.0*((data[:,18])))) - ((((0.0)) / 2.0)))) / 2.0))))) + (np.tanh(((((-1.0*(((-1.0*((data[:,22]))))))) / 2.0)))))/2.0)) / 2.0))))))
    
    def GP_class_13(self,data):
        return(0.250000*np.tanh(((((np.tanh((np.tanh((data[:,2]))))) - ((10.0)))) - (np.tanh((((((data[:,7]) - ((((((6.0)) - ((4.87243366241455078)))) - ((9.0)))))) - ((14.80352973937988281)))))))) +
                0.250000*np.tanh(((np.tanh((((((3.0)) + (data[:,1]))/2.0)))) - ((7.0)))) +
                0.250000*np.tanh(((((np.tanh((np.tanh((np.tanh((data[:,6]))))))) * 2.0)) - ((6.10337877273559570)))) +
                0.250000*np.tanh(((((((data[:,9]) + (data[:,14]))) + (data[:,14]))) + (data[:,14]))) +
                0.250000*np.tanh(((((((np.tanh(((((-1.0*(((((((data[:,2]) + ((10.0)))/2.0)) / 2.0))))) - ((13.28130435943603516)))))) + ((10.0)))) - ((13.28130435943603516)))) + ((-1.0*((data[:,5])))))) +
                0.250000*np.tanh(((((((((((data[:,20]) + (((((((((data[:,20]) * 2.0)) * 2.0)) * 2.0)) + (data[:,13]))))/2.0)) + (((data[:,9]) + (((data[:,9]) + (((data[:,6]) + (data[:,9]))))))))/2.0)) + (data[:,20]))) + (data[:,15]))) +
                0.250000*np.tanh(((((((((data[:,14]) / 2.0)) + ((((data[:,9]) + (((data[:,15]) + (((data[:,5]) + ((((((data[:,9]) + (((data[:,5]) + (data[:,14]))))) + (data[:,9]))/2.0)))))))/2.0)))) * 2.0)) + (data[:,14]))) +
                0.250000*np.tanh((((data[:,13]) + (data[:,7]))/2.0)) +
                0.250000*np.tanh(((data[:,9]) + ((((data[:,3]) + ((((data[:,21]) + (data[:,3]))/2.0)))/2.0)))) +
                0.250000*np.tanh((((data[:,2]) + (((((((((((((((data[:,20]) + (data[:,1]))) + (((data[:,2]) + (data[:,14]))))) + (data[:,2]))) + (np.tanh((data[:,2]))))/2.0)) + (((data[:,14]) + (data[:,2]))))) + (data[:,14]))/2.0)))/2.0)))
    
    def GP_class_14(self,data):
        return(0.250000*np.tanh((((((((((((5.54024744033813477)) - ((((((9.0)) * 2.0)) * 2.0)))) - (data[:,0]))) - ((((7.0)) + ((4.74105215072631836)))))) - (((((7.0)) + (((data[:,5]) * 2.0)))/2.0)))) - ((9.0)))) +
                0.250000*np.tanh(((((((((9.0)) - ((14.27298927307128906)))) - (data[:,0]))) + (np.tanh(((((12.77708148956298828)) - ((14.80560111999511719)))))))/2.0)) +
                0.250000*np.tanh(((((((0.0)) + ((-1.0*((((data[:,22]) + (((((((((13.30077362060546875)) - (np.tanh((data[:,4]))))) + (((data[:,22]) / 2.0)))/2.0)) / 2.0))))))))/2.0)) - (((np.tanh((((data[:,7]) - (data[:,7]))))) / 2.0)))) +
                0.250000*np.tanh(((((((data[:,6]) + (data[:,14]))/2.0)) + (((np.tanh((((((data[:,14]) + (data[:,14]))) + (((data[:,6]) - (((data[:,5]) + (((np.tanh((data[:,21]))) + ((((data[:,14]) + ((((data[:,14]) + (data[:,14]))/2.0)))/2.0)))))))))))) + (data[:,5]))))/2.0)) +
                0.250000*np.tanh((((((((((((((((((data[:,2]) + (data[:,16]))) + ((((((data[:,2]) / 2.0)) + (data[:,9]))/2.0)))/2.0)) + (np.tanh((data[:,10]))))/2.0)) + (data[:,9]))) + (data[:,9]))/2.0)) + (data[:,21]))) + (data[:,3]))) +
                0.250000*np.tanh((((data[:,13]) + (((((((((((data[:,3]) / 2.0)) / 2.0)) / 2.0)) * (data[:,3]))) / 2.0)))/2.0)) +
                0.250000*np.tanh(((data[:,14]) + (((((((data[:,14]) / 2.0)) + (data[:,14]))) * 2.0)))) +
                0.250000*np.tanh(data[:,9]) +
                0.250000*np.tanh(((data[:,1]) / 2.0)) +
                0.250000*np.tanh((((((data[:,2]) + ((((((data[:,13]) + (data[:,14]))) + (data[:,13]))/2.0)))) + (np.tanh((data[:,2]))))/2.0)))
    
    def GP_class_15(self,data):
        return(0.250000*np.tanh((((((((4.0)) + ((((8.0)) / 2.0)))) - ((9.56193733215332031)))) - (((((np.tanh(((13.93556308746337891)))) - (data[:,18]))) + ((13.93556308746337891)))))) +
                0.250000*np.tanh(((data[:,4]) - ((((12.76094818115234375)) - (data[:,8]))))) +
                0.250000*np.tanh((((-1.0*(((12.35446166992187500))))) - (((((12.35446166992187500)) + ((((((12.35446166992187500)) - ((-1.0*(((0.0))))))) - (data[:,0]))))/2.0)))) +
                0.250000*np.tanh(((data[:,21]) - ((14.46367263793945312)))) +
                0.250000*np.tanh(((((data[:,2]) + ((((data[:,9]) + ((-1.0*(((-1.0*(((-1.0*((((data[:,7]) * (((data[:,21]) * 2.0))))))))))))))/2.0)))) + (data[:,9]))) +
                0.250000*np.tanh(((((((((((data[:,7]) + (data[:,3]))/2.0)) + (data[:,20]))/2.0)) - ((((2.33354020118713379)) / 2.0)))) - ((((2.0)) + ((8.78930377960205078)))))) +
                0.250000*np.tanh(((((((data[:,14]) + (((((((data[:,14]) + (((data[:,15]) + (data[:,14]))))) + (((data[:,15]) + (data[:,14]))))) + (((data[:,14]) / 2.0)))))) + (data[:,14]))) + (data[:,14]))) +
                0.250000*np.tanh(((((((((data[:,13]) + (data[:,13]))) + (((data[:,13]) + (data[:,12]))))) + (data[:,9]))) + ((((((data[:,9]) + (((data[:,13]) + ((((data[:,9]) + (((((data[:,13]) * 2.0)) / 2.0)))/2.0)))))/2.0)) * 2.0)))) +
                0.250000*np.tanh((((data[:,9]) + ((((((data[:,9]) + (data[:,9]))/2.0)) / 2.0)))/2.0)) +
                0.250000*np.tanh((((((data[:,14]) + (data[:,8]))) + (data[:,14]))/2.0)))
    
    def GP_class_16(self,data):
        return(0.250000*np.tanh(((((((((((((9.37592792510986328)) - ((13.36316204071044922)))) + ((11.89122962951660156)))/2.0)) - ((((9.0)) - (((((((data[:,13]) / 2.0)) / 2.0)) + ((((-1.0*((((data[:,4]) / 2.0))))) * 2.0)))))))) - ((12.24639320373535156)))) - ((11.89122581481933594)))) +
                0.250000*np.tanh(((((np.tanh(((4.51821422576904297)))) - ((13.23712635040283203)))) - ((((((13.23712635040283203)) + (((data[:,22]) - ((((11.40539550781250000)) - (((((9.0)) + ((((((((data[:,7]) + (((data[:,17]) / 2.0)))/2.0)) * 2.0)) / 2.0)))/2.0)))))))) - (data[:,5]))))) +
                0.250000*np.tanh(((data[:,14]) - ((((4.0)) + (((((13.41770648956298828)) + ((12.42538261413574219)))/2.0)))))) +
                0.250000*np.tanh((((5.0)) - ((9.27778816223144531)))) +
                0.250000*np.tanh(((((data[:,16]) - (((((((data[:,1]) - (((np.tanh((((((((9.29507255554199219)) - (((((((14.34461498260498047)) * 2.0)) + (((((7.0)) + ((9.10683441162109375)))/2.0)))/2.0)))) + ((4.21145153045654297)))/2.0)))) * 2.0)))) - ((9.10683441162109375)))) * 2.0)))) - ((((14.34461498260498047)) * 2.0)))) +
                0.250000*np.tanh((((((((((data[:,17]) / 2.0)) + (((data[:,13]) + (data[:,9]))))/2.0)) + (((data[:,17]) + (data[:,9]))))) + (data[:,0]))) +
                0.250000*np.tanh(((data[:,13]) + (((data[:,14]) - (data[:,21]))))) +
                0.250000*np.tanh((((((data[:,14]) + (data[:,5]))/2.0)) + (((((((data[:,9]) / 2.0)) + (data[:,14]))) / 2.0)))) +
                0.250000*np.tanh((((((data[:,17]) + (data[:,9]))) + (((((data[:,17]) / 2.0)) + ((((((((((data[:,9]) + (data[:,9]))) - ((3.0)))) + (data[:,7]))) + (data[:,9]))/2.0)))))/2.0)) +
                0.250000*np.tanh(np.tanh(((((((((((((((data[:,2]) + (((((((((((data[:,22]) + (data[:,21]))/2.0)) + (data[:,2]))/2.0)) * 2.0)) + ((((((data[:,21]) * 2.0)) + (data[:,21]))/2.0)))))) + (data[:,2]))/2.0)) + (data[:,17]))/2.0)) + (np.tanh((data[:,8]))))/2.0)) + (data[:,2]))))))
    
    def GP_class_17(self,data):
        return(0.250000*np.tanh(((((data[:,14]) - ((((data[:,5]) + (((((14.43326377868652344)) + ((6.0)))/2.0)))/2.0)))) - (((((((10.0)) + ((14.84462165832519531)))/2.0)) - ((((7.0)) + (data[:,13]))))))) +
                0.250000*np.tanh((((((((((data[:,14]) + (((((((((data[:,11]) - ((9.71331787109375000)))) / 2.0)) + ((9.0)))) - ((3.0)))))/2.0)) - ((10.0)))) - ((((10.0)) + ((((9.0)) - ((((((10.0)) - ((9.0)))) / 2.0)))))))) - ((10.0)))) +
                0.250000*np.tanh((((((((((8.0)) - ((12.78277492523193359)))) - (data[:,10]))) - (np.tanh(((((((7.0)) - ((14.95321178436279297)))) - ((8.0)))))))) - ((14.95321178436279297)))) +
                0.250000*np.tanh((((7.45682907104492188)) - ((14.98023796081542969)))) +
                0.250000*np.tanh(((((((((((7.0)) + (((((np.tanh((((((((8.0)) * (data[:,8]))) + (data[:,11]))/2.0)))) / 2.0)) - ((8.0)))))/2.0)) - (data[:,20]))) - ((((8.0)) * 2.0)))) + ((8.0)))) +
                0.250000*np.tanh(((data[:,8]) + (((data[:,4]) + (((np.tanh((data[:,4]))) + (((data[:,13]) + (data[:,13]))))))))) +
                0.250000*np.tanh(data[:,14]) +
                0.250000*np.tanh(((((((((((data[:,14]) + (np.tanh((((data[:,3]) * (data[:,13]))))))) + (data[:,14]))) + (data[:,0]))) + (data[:,8]))) / 2.0)) +
                0.250000*np.tanh((((((data[:,9]) + (np.tanh(((((((data[:,19]) * (((((((data[:,1]) + (((((((data[:,21]) + (data[:,9]))/2.0)) + (data[:,6]))/2.0)))/2.0)) + (data[:,17]))/2.0)))) + (((data[:,15]) / 2.0)))/2.0)))))/2.0)) * 2.0)) +
                0.250000*np.tanh(((data[:,1]) + (((((data[:,3]) + (data[:,15]))) + (np.tanh(((((data[:,20]) + (((data[:,1]) * 2.0)))/2.0)))))))))
    
    def GP_class_18(self,data):
        return(0.250000*np.tanh((((((((((12.59485149383544922)) - ((12.59485149383544922)))) - ((12.59485149383544922)))) - ((((11.47756862640380859)) / 2.0)))) - ((12.59485149383544922)))) +
                0.250000*np.tanh((((((-1.0*(((11.26121807098388672))))) - (((((((((11.33140659332275391)) - ((-1.0*(((9.76293182373046875))))))) - (((((11.33140659332275391)) + ((8.0)))/2.0)))) + ((9.76293182373046875)))/2.0)))) - (data[:,11]))) +
                0.250000*np.tanh(((((((((((3.40250444412231445)) - (np.tanh(((14.86789989471435547)))))) - ((14.86789989471435547)))) + ((((((data[:,5]) + ((-1.0*(((((14.86789989471435547)) * 2.0))))))/2.0)) - ((14.86789989471435547)))))/2.0)) - (((((14.86789989471435547)) + (((((7.0)) + ((-1.0*((data[:,16])))))/2.0)))/2.0)))) +
                0.250000*np.tanh(((data[:,18]) - ((11.84332561492919922)))) +
                0.250000*np.tanh(((((np.tanh(((((8.96548557281494141)) * (data[:,12]))))) - ((-1.0*((((data[:,2]) * (((((data[:,4]) - ((-1.0*((data[:,9])))))) - ((9.0))))))))))) - ((8.96548557281494141)))) +
                0.250000*np.tanh(((((((10.0)) - (data[:,12]))) + (((((((((data[:,13]) - ((12.43707370758056641)))) - ((((12.43707370758056641)) - (((data[:,12]) / 2.0)))))) - ((((12.43707370758056641)) - ((((data[:,13]) + ((12.43707370758056641)))/2.0)))))) - ((12.43707370758056641)))))/2.0)) +
                0.250000*np.tanh(((((((np.tanh((((np.tanh((data[:,14]))) + (data[:,13]))))) * 2.0)) + (data[:,14]))) + (data[:,13]))) +
                0.250000*np.tanh(((np.tanh((data[:,14]))) + (data[:,14]))) +
                0.250000*np.tanh(((((data[:,9]) + (((((data[:,13]) + (data[:,4]))) + (data[:,13]))))) / 2.0)) +
                0.250000*np.tanh(((data[:,15]) - ((((data[:,3]) + ((5.0)))/2.0)))))
    
    def GP_class_19(self,data):
        return(0.250000*np.tanh((((((data[:,13]) + (data[:,13]))/2.0)) + ((((((data[:,8]) + ((((data[:,14]) + (data[:,14]))/2.0)))/2.0)) * 2.0)))) +
                0.250000*np.tanh((((data[:,3]) + (((data[:,8]) + (np.tanh(((((((-1.0*((((((9.05535793304443359)) + ((((data[:,14]) + (((((((data[:,1]) - (((data[:,12]) * 2.0)))) * ((-1.0*((data[:,17])))))) / 2.0)))/2.0)))/2.0))))) + (data[:,2]))) * 2.0)))))))/2.0)) +
                0.250000*np.tanh(((((((((data[:,3]) + (((((data[:,14]) + (data[:,14]))) * 2.0)))) + ((((data[:,14]) + (np.tanh((((data[:,14]) + (np.tanh((data[:,14]))))))))/2.0)))) * 2.0)) + ((((data[:,14]) + (data[:,2]))/2.0)))) +
                0.250000*np.tanh(((((data[:,8]) + (((data[:,4]) + (((data[:,13]) + (data[:,10]))))))) + (((((data[:,13]) + (((((data[:,13]) / 2.0)) + (data[:,9]))))) + (np.tanh(((((data[:,8]) + (data[:,8]))/2.0)))))))) +
                0.250000*np.tanh((((((data[:,19]) + (((((data[:,10]) + ((((data[:,11]) + (data[:,19]))/2.0)))) + (data[:,10]))))/2.0)) + (data[:,2]))) +
                0.250000*np.tanh((((((data[:,0]) + (((((((((data[:,0]) + (((((data[:,0]) - (data[:,11]))) / 2.0)))/2.0)) * 2.0)) + ((((((np.tanh((data[:,13]))) + (((data[:,0]) / 2.0)))/2.0)) / 2.0)))/2.0)))/2.0)) * 2.0)) +
                0.250000*np.tanh(((((-1.0*((data[:,15])))) + (((data[:,14]) + (data[:,14]))))/2.0)) +
                0.250000*np.tanh(((((((((data[:,0]) + (data[:,15]))/2.0)) + (((data[:,15]) * (data[:,15]))))/2.0)) - (data[:,11]))) +
                0.250000*np.tanh(data[:,13]) +
                0.250000*np.tanh(((((((((data[:,18]) + ((((data[:,4]) + (data[:,18]))/2.0)))) / 2.0)) + ((((data[:,18]) + (data[:,4]))/2.0)))) * (data[:,4])))    )

## Simulate Test Iterations

In [ ]:
%%time

import warnings
warnings.filterwarnings("ignore")

def simulate_iter_test_on_train_data (num_plays = 3438):
    train = pd.read_csv('../data/train.csv',  low_memory=False)
    train = train.drop ( ["Yards"], axis=1 )
    steps = min (num_plays, train.shape[0] // 22)
    cols = ["PlayId"] + ["Yards"+str(x) for x in range (-99,100)] 
    
    pred_data = np.zeros ( (steps, 200)) 
    pred_data [:,0 ] = train["PlayId"][:steps].values
    train_pred = pd.DataFrame ( pred_data, columns = cols  )
    
    for i in range (steps):
        first_row =i*22 
        df = train[first_row:first_row + 22]
        sub = train_pred [i:i+1]
        yield df, sub

gp = GP()

for n, (train,sample_prediction) in tqdm_notebook(enumerate(simulate_iter_test_on_train_data(num_plays=3438))):
    # code to test
    #simulated_test = train
    #simulated_test_prediction = sample_prediction
       
    basetable = create_features(train, deploy=True)
    basetable.drop(['GameId','PlayId'], axis=1, inplace=True)
    scaled_basetable = scaler.transform(basetable)
    
    y_pred_nn = model.predict(scaled_basetable)

    y_pred_gp = np.zeros((train.shape[0],199))
    ans = gp.GrabPredictions(scaled_basetable)
    y_pred_gp[:,96:96+20] = ans
    
    y_pred = (.6*y_pred_nn+.4*y_pred_gp)
    y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1).tolist()[0]
    
    sample_prediction.drop(['PlayId'], axis=1, inplace=True)
    preds_df = pd.DataFrame(data=[y_pred], columns=sample_prediction.columns)
    
    simulated_test_prediction = y_pred
    
print(n)

# Time for the actual submission

In [ ]:
from kaggle.competitions import nflrush
env = nflrush.make_env()
iter_test = env.iter_test()
gp = GP()
for (test_df, sample_prediction_df) in iter_test:
    test_df = test_df[test_df.NflId==test_df.NflIdRusher].copy()

    test_df['YardsFromOwnGoal'] = np.where(test_df.FieldPosition == test_df.PossessionTeam,
                                       test_df.YardLine, 50 + (50-test_df.YardLine))

    test_df[['prev_game', 'prev_play', 'prev_team', 'prev_yfog']] = test_df[
             ['GameId', 'PlayId', 'Team', 'YardsFromOwnGoal']].shift(1)

    filt = (test_df.GameId==test_df.prev_game) & (test_df.Team==test_df.prev_team) & (test_df.PlayId-test_df.prev_play<30)
    test_df.loc[filt,'est_prev_yards'] = test_df[filt]['YardsFromOwnGoal'] - test_df[filt]['prev_yfog']

    basetable = create_features(test_df, deploy=True)
    basetable.drop(['GameId','PlayId'], axis=1, inplace=True)
    scaled_basetable = scaler.transform(basetable)
    
    y_pred_nn = model.predict(scaled_basetable)

    y_pred_gp = np.zeros((test_df.shape[0],199))
    ans = gp.GrabPredictions(scaled_basetable)
    y_pred_gp[:,96:96+20] = ans
    
    y_pred = (.6*y_pred_nn+.4*y_pred_gp)
    y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1).tolist()[0]
    
    preds_df = pd.DataFrame(data=[y_pred], columns=sample_prediction_df.columns)
    env.predict(preds_df)
    
env.write_submission_file()

In [56]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

df = pd.read_csv('../data/train.csv', low_memory=False)
df_play = df[df.NflId==df.NflIdRusher].copy()

df_play['YardsFromOwnGoal'] = np.where(df_play.FieldPosition == df_play.PossessionTeam,
                                       df_play.YardLine, 50 + (50-df_play.YardLine))
df_play[['prev_game', 'prev_play', 'prev_team', 'prev_yfog', 'prev_yards']] = df_play[
        ['GameId', 'PlayId', 'Team', 'YardsFromOwnGoal', 'Yards']].shift(1)

filt = (df_play.GameId==df_play.prev_game) & (df_play.Team==df_play.prev_team) & (df_play.PlayId-df_play.prev_play<30)
df_play.loc[filt,'est_prev_yards'] = df_play[filt]['YardsFromOwnGoal'] - df_play[filt]['prev_yfog']

#plt.figure(figsize=(8,8))
#plt.title('deduced yards for %d of %d plays' % (sum(filt), len(filt)))
#plt.scatter(*zip(*df_play[['est_prev_yards', 'prev_yards']].dropna().values), alpha=0.1)
#plt.xlabel('deduced yards')
#plt.ylabel('actual yards')
#plt.show()

In [57]:
df_play.head()

,GameId,PlayId,Team,X,Y,S,A,Dis,Orientation,Dir,NflId,DisplayName,JerseyNumber,Season,YardLine,Quarter,GameClock,PossessionTeam,Down,Distance,FieldPosition,HomeScoreBeforePlay,VisitorScoreBeforePlay,NflIdRusher,OffenseFormation,OffensePersonnel,DefendersInTheBox,DefensePersonnel,PlayDirection,TimeHandoff,TimeSnap,Yards,PlayerHeight,PlayerWeight,PlayerBirthDate,PlayerCollegeName,Position,HomeTeamAbbr,VisitorTeamAbbr,Week,Stadium,Location,StadiumType,Turf,GameWeather,Temperature,Humidity,WindSpeed,WindDirection,YardsFromOwnGoal,prev_game,prev_play,prev_team,prev_yfog,prev_yards,est_prev_yards
18,2017090700,20170907000118,home,78.75,30.53,3.63,3.35,0.38,161.98,245.74,2543773,James White,28,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,5-10,205,02/03/1992,Wisconsin,RB,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW,35,NaN,NaN,NaN,NaN,NaN,NaN
40,2017090700,20170907000139,home,71.07,27.16,3.06,2.41,0.34,210.70,312.20,2543773,James White,28,2017,43,1,13:52:00,NE,1,10,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:27.000Z,2017-09-08T00:44:26.000Z,3,5-10,205,02/03/1992,Wisconsin,RB,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW,43,2.017091e+09,2.017091e+13,home,35.0,8.0,8.0
62,2017090700,20170907000189,home,48.66,19.11,5.77,2.42,0.60,140.82,221.96,2543773,James White,28,2017,35,1,13:02:00,NE,1,10,KC,0,0,2543773,SINGLEBACK,"1 RB, 1 TE, 3 WR",7.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:45:17.000Z,2017-09-08T00:45:15.000Z,5,5-10,205,02/03/1992,Wisconsin,RB,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW,65,2.017091e+09,2.017091e+13,home,43.0,3.0,NaN
84,2017090700,20170907000345,home,15.53,25.36,4.45,3.20,0.46,186.22,275.44,2539663,Mike Gillislee,35,2017,2,1,12:12:00,NE,2,2,KC,0,0,2539663,JUMBO,"6 OL, 2 RB, 2 TE, 0 WR",9.0,"4 DL, 4 LB, 3 DB",left,2017-09-08T00:48:41.000Z,2017-09-08T00:48:39.000Z,2,5-11,210,11/01/1990,Florida,RB,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW,98,2.017091e+09,2.017091e+13,home,65.0,5.0,NaN
98,2017090700,20170907000395,away,29.99,27.12,3.90,2.53,0.44,34.27,157.92,2557917,Kareem Hunt,27,2017,25,1,12:08:00,KC,1,10,KC,7,0,2557917,SHOTGUN,"1 RB, 3 TE, 1 WR",7.0,"3 DL, 2 LB, 6 DB",right,2017-09-08T00:53:14.000Z,2017-09-08T00:53:13.000Z,7,5-11,216,08/06/1995,Toledo,RB,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW,25,2.017091e+09,2.017091e+13,home,98.0,2.0,NaN
